# Object Relation Mapping of a Fitbit Dataset 

### Importing libraries and define file paths

In [1]:
from pandas import read_json  as json_to_table
from pandas import read_excel as excel_to_table
from pandas import read_csv as csv_to_table
from pony.orm import Database, Required, Optional, PrimaryKey, Set, select, avg, count
from datetime import datetime
from pandas import DataFrame  as Table
from pandas import merge as Merge_df
from pandas import date_range, concat
from functools import reduce
import jupyter_dash
from dash import html
import dash_pivottable
import dash

import logging

In [2]:
# #File path definition LINDA
# PARTECIPANTS_FILE = 'participant-overview.xlsx' 
# PMDATA = 'pmdata\\'
# CALORIES = '\\fitbit\calories.json'
# DISTANCE = '\\fitbit\distance.json'
# EXERCISE = '\\fitbit\exercise.json'
# HEART_RATE = '\\fitbit\heart_rate.json'
# LIGHTLY_ACTIVE_MIN = '\\fitbit\lightly_active_minutes.json'
# MODERATE_ACTIVE_MIN = '\\fitbit\moderately_active_minutes.json'
# RESTING_HEART = '\\fitbit\\resting_heart_rate.json'
# SEDENTARY_MIN = '\\fitbit\sedentary_minutes.json'
# SLEEP_SCORE = '\\fitbit\sleep_score.csv'
# SLEEP = '\\fitbit\sleep.json'
# STEPS = '\\fitbit\steps.json'
# TIME_HEART_ZONE_RATE = '\\fitbit\\time_in_heart_rate_zones.json'
# VERY_ACTIVE_MIN = '\\fitbit\\very_active_minutes.json'
# PATH_LIST_JSON = [CALORIES, DISTANCE, EXERCISE, HEART_RATE, LIGHTLY_ACTIVE_MIN, 
#         MODERATE_ACTIVE_MIN, RESTING_HEART, SEDENTARY_MIN, SEDENTARY_MIN,
#         SLEEP, STEPS, TIME_HEART_ZONE_RATE, VERY_ACTIVE_MIN]
# LOG = '../1.Data Warehouse OLAP systems for healthcare/'

#File path definition ALBE
PARTECIPANTS_FILE = 'datasets/pmdata/participant-overview.xlsx' 
PMDATA = 'datasets/pmdata\\'
CALORIES = '\\fitbit\calories.json'
DISTANCE = '\\fitbit\distance.json'
EXERCISE = '\\fitbit\exercise.json'
HEART_RATE = '\\fitbit\heart_rate.json'
LIGHTLY_ACTIVE_MIN = '\\fitbit\lightly_active_minutes.json'
MODERATE_ACTIVE_MIN = '\\fitbit\moderately_active_minutes.json'
RESTING_HEART = '\\fitbit\\resting_heart_rate.json'
SEDENTARY_MIN = '\\fitbit\sedentary_minutes.json'
SLEEP_SCORE = '\\fitbit\sleep_score.csv'
SLEEP = '\\fitbit\sleep.json'
STEPS = '\\fitbit\steps.json'
TIME_HEART_ZONE_RATE = '\\fitbit\\time_in_heart_rate_zones.json'
VERY_ACTIVE_MIN = '\\fitbit\\very_active_minutes.json'
PATH_LIST_JSON = [CALORIES, DISTANCE, EXERCISE, HEART_RATE, LIGHTLY_ACTIVE_MIN, 
        MODERATE_ACTIVE_MIN, RESTING_HEART, SEDENTARY_MIN, SEDENTARY_MIN,
        SLEEP, STEPS, TIME_HEART_ZONE_RATE, VERY_ACTIVE_MIN]
LOG = '../1.Data Warehouse OLAP systems for healthcare/'


## DataBase 

In [3]:
#Connessione al DB !RIcordarsi di far andare il server!
DB = Database()
DB.bind(provider='postgres', 
        database='postgres',
        user='postgres', 
        port='5433',  #Linda: 5432
        host='localhost',
        password='admin'
        )

## LogDebug

In [4]:
from datetime import datetime as a
import os, shutil

log_date_filename = 'HIS_'+a.today().strftime('%Y%m%d_%H%M%S')+'.log'
log_date_filename

path = os.getcwd()
pathLogs = path+'\\LOGS'
if not os.path.exists(path):
        d = os.makedirs(f'{path}\\LOGS')
else: d = pathLogs

logging.basicConfig(
                    filename = f'{d}/{log_date_filename}'
                    , filemode='w'
                    , format = "[%(asctime)s] %(levelname)s - %(message)s"
                    , level = logging.DEBUG
                    , force = True
            )
#basicConfig setta le configuarioni base del file dove salvare i log del codice
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  #setta il livello di logging 

logger.debug("********* INIZIO LOG *********")

### Table creation

**PARTECIPANT**

In [5]:

#Si va a creare la classe Partecipant che corrisponde alla tabella a db. 
class Partecipant(DB.Entity):
    name = PrimaryKey(str)
    age  = Optional(int)
    height = Optional(int)
    gender = Optional(str)
    Max_heart_rate = Optional(int)
    group = Optional(str)
    base_line_cal = Optional(float)
    BPM = Set('BPM')
    calories = Set('Calories')
    exercise = Set('Exercises')
    sleep = Set('Sleep')
    steps = Set('Steps')
    distance = Set('Distances')
    day = Set('Day')

#Inizio implememtazione delle funzioni che verranno utilizzate poi per elaborare i dati


     # Controlla la presenza dei partecipanti in un specifico giorno.
    @classmethod
    def add_partecipant_day(self, name, day):
        
        #logger.debug('Inizio metodo {classmethod}')
        logger.error('Inizio metodo {classmethod} error')
        if not self.exists(name):  #se non esiste richiama la funzione che permette di cresre un nuovo partecipante
            print('Partecipant {name} does not exists')
            print('Lancio metodo di creazione partecipant')
            self.create_new_partecipant(name)
        else:
            print('Partecipant already exists')
        
        day = datetime.strptime(day, "%y-%m-%d") #cast che trasforma da stringa a datetime


        # Creazione funzioni che importano i dati dal DB
        #
        # Calories
        self.createCalories(date = day, file = json_to_table("".join([PMDATA, name, CALORIES])), name = name)
        # Distances
        self.createDistances(date = day, file = json_to_table("".join([PMDATA, name, DISTANCE])), name = name)
        # BPM
        BPM = self.createBPM(date = day, file = json_to_table("".join([PMDATA, name, HEART_RATE])), name = name)
        # Exercise
        self.createExercise(date = day, file = json_to_table("".join([PMDATA, name, EXERCISE])), name = name)
        # Sleep
        self.createSleep(date = day, file = json_to_table("".join([PMDATA, name, SLEEP])), name = name)
        # StepsSLEEP
        self.createSteps(date = day, file = json_to_table("".join([PMDATA, name, STEPS])), name = name)
        # Day
        self.createDay(date = day, name = name)
    

    #Funzione per creare un nuovo partecipanete se non esiste già
    @classmethod
    def create_new_partecipant(self, name):

        logger.debug('Inizio metodo create_new_partecipant()')
        logger.debug('Lettura file excel e salvataggio in df')
        df = excel_to_table(PARTECIPANTS_FILE, header=1, engine='openpyxl')  #permette di leggere il file excel e salvarlo in un dataframe
        
        logger.debug('Lettura singoli dati per colonna')
        #Si scorre e si estraggono i dati dal dataframe appena creato, a partire dal nome del partecipante
        age = (list(df[ df["Partecipant ID"] == name].Age)[0])
        logger.debug("Age = %s",age, exc_info=True)
        
        height =  list(df[ df["Partecipant ID"]  == name].Height)[0]
        logger.debug("Height = %s",height, exc_info=1)

        gender = list(df[ df["Partecipant ID"]  == name].Gender)[0]
        logger.debug("gender = %s",gender, exc_info=1)
        max_heart_rate = (list(df[ df["Partecipant ID"]  == name].Max_heart_rate)[0])
        if max_heart_rate is None:
            max_heart_rate = 0
        logger.debug("max_heart_rate = %s",{max_heart_rate}, exc_info=1)
        group = (list(df[ df["Partecipant ID"]  == name].A_or_B_person))[0]
        logger.debug("group = %s",group, exc_info=1)

        partecipant = Partecipant(
            name = name,
            age = age,
            height = height,
            gender = gender,
            Max_heart_rate = max_heart_rate,
            group = group
        )
        
        logging.shutdown()

        return partecipant


    #Funzione che va ad inserire tutute le calories del giorno passato in input
    @classmethod
    def createCalories(self, date, file, name):
     
        #date  = datetime.strptime(date, "%Y-%m-%d")  NEcessario per le singole chiamate se si vogliono testare
        print(date)
        #Funzione che ritorna una lista di valori filtrati per data. 
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        print(a)
        list_filtered_date = list(a)
        print(list_filtered_date)
        base_cal = min(file.value) #si va a prendere il min valore delle calorie presente nel file
        print(base_cal)
        # p = Partecipant(name = name) #csi crea un istanza di tipo Partecipant utilizzando il nome passato in inpurt
        # print(p)
        Partecipant.base_line_cal = base_cal

        for date in list_filtered_date:
            if float(list(file[ file["dateTime"] == date].value)[0]) > base_cal:  #Se il valore delle calorie per la data è maggiore della base cal
                Calories(partecipant = name,   #si crea un istanza di calories con il nome, data e e il valore della caloria filtrato
                         vt = date,
                         value = float(list(file[ file["dateTime"] == date].value)[0])
                )


   #Funzione che va a inserire e creare tutte le distanze della data passata in input
    @classmethod
    def createDistances(self, date, file, name):
        
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        list_filtered_date = list(a)
        
        for date in list_filtered_date:
            Distances(partecipant = name,
                    value = float(list(file[ file["dateTime"] == date].value)[0]),
                    vt = date
            )

#Funzione che va a inserire e creare tutti i valori di BPM  della data passata in input
    @classmethod
    def createBPM(self, date, file, name): 
        
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        list_filtered_date = list(a)
        
        for date in list_filtered_date:
            BPM(partecipant = name, 
                value = list(file[ file["dateTime"] == date].value)[0].get("bpm"),
                vt = date, 
                confidence = list(file[ file["dateTime"] == date].value)[0].get("confidence")
                )
    


    @classmethod
    def createExercise(self, date, file, name):
        #iterazione per ogni valore nella colonna facedno il cast per trasforamre da stringa a datetime. I risultati vengono salvati una lista
        list_day = [datetime.strptime(day, "%Y-%m-%d %H:%M:%S") for day in list(file.startTime)]
        #function that return a list of values filtered by date 
        a = filter(lambda x: 
            (x >= date and x< datetime(date.year, date.month, date.day+1)) , 
            list_day)
        lista_date_filtrate = list(a)
       
         #loop lista date creata sopra. dove viene ti trasforma il datetime in stringa e si va a crare l'istanza Exercises pasadno 
         #i valori letti dal file in base alla data filtrata sopra.
        for date in lista_date_filtrate:
            date = date.strftime("%Y-%m-%d %H:%M:%S")   
            Exercises(
                vt = date,
                partecipant = name,
                activityName = list(file[ file["startTime"] == date].activityName)[0],
                avg_BPM = list(file[ file["startTime"] == date].averageHeartRate)[0],
                calories = list(file[ file["startTime"] == date].calories)[0],
                duration = list(file[ file["startTime"] == date].duration)[0],
                steps = list(file[ file["startTime"] == date].steps)[0],
                elevationGain = list(file[ file["startTime"] == date].elevationGain)[0]
            )

    @classmethod
    def createSleep(self, date, file, name):
        list_startTime = [datetime.strptime(day, "%Y-%m-%d %H:%M:%S") for day in list(file.startTime)]

        #for date in list_startTime:         
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list_startTime
                )
        list_filtered_date = list(a)
      #  print(list_filtered_date)

        for date in list_filtered_date:
            date = datetime.strftime(date, "%Y-%m-%d %H:%M:%S")
            
            Sleep(
                vt = date,
                partecipant = name,
                duration = list(file[ file["startTime"] == date].duration)[0]/60000,
                endTime = list(file[ file["startTime"] == date].endTime)[0],
                minutesAsleep = list(file[ file["startTime"] == date].minutesAsleep)[0],
                types = list(file[ file["startTime"] == date].type)[0],
                efficiency = list(file[ file["startTime"] == date].efficiency)[0]
            )

    @classmethod
    def createSteps(self, date, file, name):
        # function that create/insert BPM of the specific day
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        list_filtered_date = list(a)
        
        for date in list_filtered_date:
            Steps(
                partecipant = name,
                value = list(file[ file["dateTime"] == date].value)[0],
                vt = date
            )

    @classmethod
    def createDay(self, name, date):
        lightly_active_minutes_file =  json_to_table("".join([PMDATA, name, LIGHTLY_ACTIVE_MIN]))
        moderately_active_minutes_file = json_to_table("".join([PMDATA, name, MODERATE_ACTIVE_MIN]))
        resting_heart_rate_file = json_to_table("".join([PMDATA, name, RESTING_HEART]))
        sedentary_minutes_file = json_to_table("".join([PMDATA, name, SEDENTARY_MIN]))
        very_active_minutes_file = json_to_table("".join([PMDATA, name, VERY_ACTIVE_MIN]))
        time_in_heart_rate_zones_file = json_to_table("".join([PMDATA, name, TIME_HEART_ZONE_RATE]))
        sleep_score_file = csv_to_table("".join([PMDATA, name, SLEEP_SCORE]))
        IndexDay = [ list(sleep_score_file[sleep_score_file['timestamp'] == i].index.values)[0] for i in sleep_score_file['timestamp'] if datetime.strptime(i[:-10], "%Y-%m-%d") == date ]
    #    'overall_score', 'composition_score',
    #    'revitalization_score', 'duration_score', 'deep_sleep_in_minutes',
    #    'resting_heart_rate', 'restlessness']
        Day(
            partecipant = name,
            vt = date,
            lightly_active_minutes = (list(lightly_active_minutes_file[ lightly_active_minutes_file["dateTime"] == date]).value)[0],
            moderately_active_minutes = (list(moderately_active_minutes_file[ moderately_active_minutes_file["dateTime"] == date]).value)[0],
            sedentary_minutes = (list(sedentary_minutes_file[ sedentary_minutes_file["dateTime"] == date]).value)[0],
            very_active_minutes = (list(very_active_minutes_file[ very_active_minutes_file["dateTime"] == date]).value)[0],
            overall_score = sleep_score_file.loc[IndexDay[0]].overall_score,
            composition_score = sleep_score_file.loc[IndexDay[0]].composition_score,
            revitalization_score = sleep_score_file.loc[IndexDay[0]].revitalization_score,
            duration_score = sleep_score_file.loc[IndexDay[0]].revitalization_score,
            deep_sleep_in_minutes = sleep_score_file.loc[IndexDay[0]].deep_sleep_in_minutes,
            resting_heart_rate = (list(resting_heart_rate_file[ resting_heart_rate_file["dateTime"] == date]).value)[0].get("value"),

            #use .get to access to children attributes of the xml
            time_in_heart_rate_BELOW_DEFAULT_ZONE_1 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("BELOW_DEFAULT_ZONE_1"),
            time_in_heart_rate_zone_1 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("IND_DEFAULT_ZONE_1"),
            time_in_heart_rate_zone_2 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("IND_DEFAULT_ZONE_2"),
            time_in_heart_rate_zone_3 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("IND_DEFAULT_ZONE_3")
        )
 
    @classmethod
    def checkPartecipantExists(self, name):
        return not(Partecipant.get(name = name) == None)
    
    @classmethod
    def checkExistsDate(self, part, date):
        return not(Calories.get(vt = date, partecipant = part) == None)
    
    @classmethod
    def time_Aggregation(self, vts, vte, minutes):
        vts = datetime.strptime(vts, "%Y-%m-%d")
        vte = datetime.strptime(vte, "%Y-%m-%d")
        # calculate total minutes in a day and divide them by the desired minutes
        num_rows = int((24*60)/minutes)
        print(num_rows, vts, vte)
        partecipants = list(select(p.name for p in Partecipant))
        print(partecipants)

        # check for the presence in the db of the data. If not data will be loaded to db
        for day in date_range(start = vts, end = vte).to_pydatetime().tolist():
            for p in partecipants:
                if not self.checkExistsDate(part = p, date = day):
                    print(day, ' day not exist in db', p)
                    partecipants.remove(p)
        print(partecipants)

        df = Table([])

        if partecipants != []:
            for p in partecipants:
                df_BPM = self.create_df_timeAggregation_BPM( part = p, num_tuples = num_rows, vts = vts,  vte = vte )
                df_calo = self.create_df_timeAggregation_Calories( part = p, num_tuples = num_rows, vts = vts,  vte = vte )
                df_dis = self.create_df_timeAggregation_Distances( part = p, num_tuples = num_rows, vts = vts,  vte = vte )
                df_st = self.create_df_timeAggregation_Steps( part = p, num_tuples = num_rows, vts = vts,  vte = vte )
                df_exer = self.create_df_timeAggregation_Exercise( part = p, num_tuples = num_rows, vts = vts,  vte = vte )
                 
                list_df = [df_BPM, df_calo, df_dis, df_st, df_exer]            
                list_df = reduce(lambda  left,right:   #reduce funcction concatena iterativamente in modo orizzontale i dataframe richiamando la funzionù
                                 #concat. Questo vieen specificato mettendo axis = 1 che specifica che la concatenazione avviene per colonne.
                                 concat([left,right], axis=1), list_df
                                 ).fillna('no value') #utilizzato per per sostituire i valori mancantin con la dicitura 'no value'
                
                df = concat([df, list_df]) #concatena i due dataframe verticalmente. Si vanno quindi ad aggiungere alle righe di df le righe di list_df


#Funzione per creare un dataframe che aggrega i dati di BPM per un dato intervallo temporale
    @classmethod
    def create_df_timeAggregation_BPM( self, part, num_tuples, vts, vte):  #part nome partecipante, data inizio e fine
        df = []
        # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((bpm.vt, bpm.value) 
                   for bpm in BPM 
                   if (bpm.vt >= vts and bpm.vt <= vte and part == str(bpm.partecipant.name))
                   ).order_by(1)         
        
        v = Table(v, columns = ['vt', 'value']) #Istazna Table dove venogono salvati i dati appena estratti e messi in due colonne.

        for i in range(0, len(v)-1, int(len(v)/num_tuples)):                  
            end = i + int(len(v)/num_tuples)
            if end >= len(v):
                end = len(v)-1
              
            df.append([int(v['value'].loc[i:end].mean()), part])#concatena a quanto già presente in df la media dei valori da i a
            #end appena calcolato e lo associa al partecipante
        
        df = Table(df, columns= [ 'heartRate', 'Partecipant'])
        return df 
        #s = select((bpm.vt, bpm.value) for bpm in BPM)


    @classmethod
    def create_df_timeAggregation_Calories( self, part, num_tuples, vts, vte):
        date_list = date_range(start=vts, end=vte, freq='Min').to_pydatetime().tolist()#date_range è una funzione pandas che va a crare un range temporale a partire da
                     #le validtimestart e valide time end con frequenza di 1 minuto. to_pydatetime è per converitre il risultato in una lista python
   
        df = Table([[day] for day in date_list], columns=['vt'])

        # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((cal.vt, cal.value) 
                   for cal in Calories 
                   if (cal.vt >= vts and cal.vt <= vte and part == str(cal.partecipant.name))
                   ).order_by(1) 
        
        v = Table(v, columns = ['vt', 'value'])        
    
        v = Merge_df(left=df,right= v, how = 'left', on = ['vt'] )   # merge di v con df in base alla colonna vt 

        df = []     
        for i in range(0, len(v)-1, int(len(v)/num_tuples)):                 
            end = i + int(len(v)/num_tuples)
    
            df.append([ v['value'].loc[i:end].sum()])   # somma dei valori da i a end e concatenati al dataframe
        
        df = Table(df, columns= ['calories'])
        return df
    
    @classmethod
    def create_df_timeAggregation_Distances( self, part, num_tuples, vts, vte):
        df = [] 
       # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((d.vt, d.value) 
                   for d in Distances 
                   if (d.vt >= vts and d.vt <= vte and part == str(d.partecipant.name))
                   ).order_by(1) 
        
        v = Table(v, columns = ['validTime', 'value'])
       #Verificare se necessario trasformare in table
        #print(c)            
        for i in range(0, len(v), int(len(v)/num_tuples)):                 
            end = i + int(len(v)/num_tuples)
            if end >= len(v):
                end = len(v)-1
            #method to get the sum of the distance done 
            df.append([int(v['value'].loc[i:end].sum())])   #testare con questo codice = df.append([int(v[i:end]['value'].sum())]) 
        
        df = Table(df, columns= ['distances'])
        return df
    
    
    @classmethod
    def create_df_timeAggregation_Steps( self, part, num_tuples, vts, vte):
        df = [] 
        # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((s.vt, s.value) 
                   for s in Steps 
                   if (s.vt >= vts and s.vt <= vte and part == str(s.partecipant.name))
                   ).order_by(1)         
        
        v = Table(v, columns = ['validTime', 'value'])       
        #print(c)            
        for i in range(0, len(v), int(len(v)/num_tuples)):                 
            end = i + int(len(v)/num_tuples)                
            if end >= len(v):
                end = len(v)-1      
            df.append([ int(v['value'].loc[i:end].sum())])   
        
        df = Table(df, columns= [ 'steps'])
        return df
    
    @classmethod
    def create_df_timeAggregation_Exercise( self, part, num_tuples, vts, vte):
    
        date_list = date_range(start=vts,end=vte, freq='Min').to_pydatetime().tolist()#date_range è una funzione pandas che va a crare un range temporale a partire da
                     #le validtimestart e valide time end con frequenza di 1 minuto. to_pydatetime è per converitre il risultato in una lista python
        df = Table([day for day in date_list], columns= ['validTime']) 

        tmp_list = []
        for i in range(0, len(df)-1, int(len(df)/num_tuples)):         
            tmp_list.append([ df['validTime'].loc[i], False, False, False, False, False])

        df = Table(tmp_list, columns=['validTime', 'walking', 'swimming', 'running', 'cycling', 'treadmill' ])       
        
        #take all the element between the vts and vte  sort the data in order of date
        v = select((e.vt, e.activity_type) 
                   for e in Exercises
                     if (e.vt >= vts and e.vt <= vte and part == str(e.partecipant.name))
                    ).order_by(1)         
        v = Table(v, columns = ['validTime', 'activity_type'])
                    
        for i in range(0, len(v)): 
            for j in range(0, len(df)-1):
                #chidere al prof?
                if  v["validTime"].loc[i] >= df["validTime"].loc[j] and v["validTime"].loc[i] <= df["validTime"].loc[j+1] :
                    if str(v['activity_type'].loc[i]) == 'Walk':
                        df['walking'].loc[j] = True
                    elif str(v['activity_type'].loc[i]) == 'Swim' :
                        df['walking'].loc[j] = True
                    elif str(v['activity_type'].loc[i])  == 'Run' :
                        df['walking'].loc[j] = True
                    elif str(v['activity_type'].loc[i])  == 'Cycle' :
                        df['walking'].loc[j] = True
                    elif str(v['activity_type'].loc[i]) == 'Treadmill':
                        df['walking'].loc[j] = True
        
        return df



class Calories(DB.Entity):
    value = Required(float)
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)

class Distances(DB.Entity):
    value = Required(float)
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(vt, partecipant)

class BPM(DB.Entity):
    vt = Required(datetime)
    value = Optional(int)
    confidence = Optional(int)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)

class Exercises(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    activityName = Required(str)
    avg_BPM = Optional(int)
    calories = Optional(int)
    duration = Optional(int)
    steps = Optional(int)
    elevationGain = Optional(float)
    PrimaryKey(vt, partecipant)

class Sleep(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    duration = Optional(int)
    endTime = Optional(datetime)
    minutesAsleep = Optional(int)
    types = Optional(str)
    efficiency = Optional(int)
    PrimaryKey(partecipant, vt)

class Steps(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    value = Required(int)   
    PrimaryKey(vt, partecipant) 


class Day(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)
    n_lightly_active_minutes = Optional(int)
    n_moderately_active_minutes = Optional(int)
    f_resting_heart_rate = Optional(float)
    n_sedentary_minutes = Optional(int)
    n_very_active_minutes = Optional(int)
    n_overall_score = Optional(int)
    n_composition_score = Optional(int)
    n_duration_score = Optional(int)
    n_deep_sleep_in_minutes = Optional(int)
    f_time_in_heart_rate_BELOW_DEFAULT_ZONE_1 = Optional(float)
    f_time_in_heart_rate_zone_1 = Optional(float)
    f_time_in_heart_rate_zone_2 = Optional(float)
    f_time_in_heart_rate_zone_3 = Optional(float)
        

In [6]:
DB.commit() 

In [7]:
DB.generate_mapping(create_tables=True)

In [ ]:
# # query = select((bpm.vt, bpm.value) for bpm in BPM)
# # query = select((p.name) for p in Partecipant)
# # query[:]

# name = 'p01'
# df = excel_to_table(PARTECIPANTS_FILE, header=1, engine='openpyxl')
# # list(df[ df["Partecipant ID" == 'p01']].Age)[0]
# df [ df["Partecipant ID"] == name].Age
# list(df[ df["Partecipant ID"]  == name].Gender)[0]

In [9]:
Partecipant.create_new_partecipant('p04')
DB.commit()

In [15]:
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createCalories(date = date, file = json_to_table("".join([PMDATA, 'p01', CALORIES])), name = 'p01')
# DB.commit()

2019-11-01 00:00:00
[Timestamp('2019-11-01 00:00:00'), Timestamp('2019-11-01 00:01:00'), Timestamp('2019-11-01 00:02:00'), Timestamp('2019-11-01 00:03:00'), Timestamp('2019-11-01 00:04:00'), Timestamp('2019-11-01 00:05:00'), Timestamp('2019-11-01 00:06:00'), Timestamp('2019-11-01 00:07:00'), Timestamp('2019-11-01 00:08:00'), Timestamp('2019-11-01 00:09:00'), Timestamp('2019-11-01 00:10:00'), Timestamp('2019-11-01 00:11:00'), Timestamp('2019-11-01 00:12:00'), Timestamp('2019-11-01 00:13:00'), Timestamp('2019-11-01 00:14:00'), Timestamp('2019-11-01 00:15:00'), Timestamp('2019-11-01 00:16:00'), Timestamp('2019-11-01 00:17:00'), Timestamp('2019-11-01 00:18:00'), Timestamp('2019-11-01 00:19:00'), Timestamp('2019-11-01 00:20:00'), Timestamp('2019-11-01 00:21:00'), Timestamp('2019-11-01 00:22:00'), Timestamp('2019-11-01 00:23:00'), Timestamp('2019-11-01 00:24:00'), Timestamp('2019-11-01 00:25:00'), Timestamp('2019-11-01 00:26:00'), Timestamp('2019-11-01 00:27:00'), Timestamp('2019-11-01 00:28

In [ ]:
#LINDA
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createBPM(date = date, file = json_to_table("".join([PMDATA, 'p01', HEART_RATE])), name = 'p01')
DB.commit()

In [ ]:
#LINDA
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createDistances(date = date, file = json_to_table("".join([PMDATA, 'p01', DISTANCE])), name = 'p01')
DB.commit()

In [14]:
#LINDA
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createExercise(date = date, file = json_to_table("".join([PMDATA, 'p01', EXERCISE])), name = 'p01')
DB.commit()

In [ ]:
#LINDA
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createSleep(date = date, file = json_to_table("".join([PMDATA, 'p01', SLEEP])), name = 'p01')
DB.commit()

In [8]:
#LINDA
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createSteps(date = date, file = json_to_table("".join([PMDATA, 'p01', STEPS])), name = 'p01')
DB.commit()

ERDiagramError: Mapping is not generated for entity 'Steps'

In [7]:
#LINDA
date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createDay(date = date, name = 'p01')
DB.commit()

AttributeError: 'list' object has no attribute 'value'

In [ ]:
Partecipant.time_Aggregation(vts = "2019-11-01" , vte= "2019-11-08", minutes= 120)

In [8]:
Partecipant.add_partecipant_day(name ='p04',day= "2019-11-10")

TypeError: Lambda function is expected. Got: p04

In [ ]:
vts = "2019-11-01"
vts = datetime.strptime(vts, "%Y-%m-%d")
vte= "2019-11-08"
vte = datetime.strptime(vte, "%Y-%m-%d")
v = select((cal.vt, cal.value) 
                   for cal in Calories 
                   if (cal.vt >= vts and cal.vt <= vte and 'p01' == str(cal.partecipant.name))
                   ).order_by(1) 
v[:]

# v1 = Table(v, columns = ['vt', 'value'])
# v1

In [ ]:
file_path = 'C:\\Users\\39349\\Downloads\\calories.json'
date = "2019-11-01 00:00:00"
Partecipant.createCalories( date=date , file= file_path , name='p02')